In [1]:
import pandas as pd
import numpy as np
realtor = pd.read_csv('realtor-data.csv')
realtor.head()

# O objetivo é verificar se o valor na coluna 'house_size' é nulo (missing) ou não. A função np.where é usada para isso. 
# Ela recebe três argumentos: a primeira é uma condição (no caso, realtor['house_size'].isna() verifica se 'house_size' é nulo),
# a segunda é o valor a ser atribuído se a condição for verdadeira (1 no seu caso, indicando que a 'house_size' está faltando),
# e a terceira é o valor a ser atribuído se a condição for falsa (0 no seu caso, indicando que a 'house_size' não está faltando).
# valores missing
# realtor['hs_missing'] = np.where(realtor['house_size'].isna(), 1, 0)

# criando a coluna hs_missing que contém quais campos da coluna 'house_size' são nulos (em forma de número - astype())
realtor['hs_missing'] = realtor['house_size'].isnull().astype(int)

# criando as features
features = (realtor
            .groupby(['city', 'state'])
            .agg({ 'house_size': ['max', 'min', 'mean'], 'hs_missing':'mean'})
            .reset_index()
            )

features.columns = ['city', 'state', 'hs_max', 'hs_min', 'hs_mean', 'hs_missing']
features.head()


,city,state,hs_max,hs_min,hs_mean,hs_missing
0,Acton,Massachusetts,99999.0,918.0,4940.695652,0.157509
1,Adams,Massachusetts,9516.0,640.0,2332.830078,0.316422
2,Adjuntas,Puerto Rico,1527.0,920.0,1214.303030,0.000000
3,Agawam,Massachusetts,3200.0,676.0,1675.292230,0.070644
4,Aguada,Puerto Rico,6000.0,600.0,2139.268116,0.471264


In [3]:
# realtor['hs_missing'] = realtor['house_size'].isnull().astype(int)
# preço maximo pelo acre min
# acre agg('{acre_lot': 'min'})

# criando as features
features2 = (realtor
            .groupby(['city', 'state'])
            .agg({'acre_lot': ['min'], 'price': ['max']})
            .reset_index()
            )

features2.columns = ['city', 'state','acre_lot_min', 'price_max']

# fazendo porcentagem
features2['acre_min_max_price'] = (features2['price_max']/features2['acre_lot_min']).round(1)

# quando acre_lot == 0 adicionar o missing simbolico de -1
#  verifica se o valor de 'acre_lot_min' é igual a zero em alguma linha e, se for o caso, atribui o valor -1 à coluna 'rz_price_acre_lot' para essa linha. 
# Isso é feito para tratar o caso em que o tamanho mínimo do lote de terreno é zero, evitando divisão por zero.
features2.loc[features2['acre_lot_min'] ==0, 'acre_min_max_price'] = -1

features2.drop(columns=['price_max', 'acre_lot_min'])

features2.head()

,city,state,acre_lot_min,price_max,acre_min_max_price
0,Acton,Massachusetts,0.15,13750000.0,91666666.7
1,Adams,Massachusetts,0.02,495000.0,24750000.0
2,Adjuntas,Puerto Rico,0.08,105000.0,1312500.0
3,Agawam,Massachusetts,0.11,779900.0,7090000.0
4,Aguada,Puerto Rico,0.07,10000000.0,142857142.9


In [4]:
# concatenando features
features_total = features.merge(features2, on=['city', 'state'], how='left')
features_total

,city,state,hs_max,hs_min,hs_mean,hs_missing,acre_lot_min,price_max,acre_min_max_price
0,Acton,Massachusetts,99999.0,918.0,4940.695652,0.157509,0.15,13750000.0,91666666.7
1,Adams,Massachusetts,9516.0,640.0,2332.830078,0.316422,0.02,495000.0,24750000.0
2,Adjuntas,Puerto Rico,1527.0,920.0,1214.303030,0.000000,0.08,105000.0,1312500.0
3,Agawam,Massachusetts,3200.0,676.0,1675.292230,0.070644,0.11,779900.0,7090000.0
4,Aguada,Puerto Rico,6000.0,600.0,2139.268116,0.471264,0.07,10000000.0,142857142.9
...,...,...,...,...,...,...,...,...,...
577,Worcester,Massachusetts,14532.0,456.0,2469.285381,0.087511,0.04,1795000.0,44875000.0
578,Worthington,Massachusetts,14240.0,2180.0,6878.363636,0.629213,2.77,2995000.0,1081227.4
579,Wrentham,Massachusetts,3872.0,1044.0,2290.941176,0.227273,0.25,1875000.0,7500000.0
580,Yabucoa,Puerto Rico,4902.0,849.0,1634.038462,0.434783,0.05,380000.0,7600000.0
